# Acoustic Absorption 2D Example

This notebook demonstrates a 2D acoustic wave simulation for a homogeneous absorbing medium. We'll guide you through the setup process step by step.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import sys
import os
module_path = os.path.abspath(os.path.join( '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import ultrawave
from ultrawave.lib.model_2d import Model

## GPU acceleration

To use GPU to accelerate this simulation, you need to have NVIDIA HPC SDK installed, and run the following code.

If you don't want to use GPU, just skip this part. The code will automatically ran on CPU.

In [ ]:
from devito import configuration
configuration['platform'] = 'nvidiaX'
configuration['compiler'] = 'pgcc'
configuration['language'] = 'openacc'

## Define the model

The model encapsulates the simulation grid and medium properties:

- **Grid Setup**: We start by defining a grid with specified spacing and shape.
- **Simulated Medium**: A homogeneous medium with a sound speed of $1500 \text{ m/s}$ and a density of $1000 \text{ kg/m}^3$.


In [ ]:
# Define grid spacing in [m]
spacing = (5.e-5, 5.e-5) # [m]

# The number of grid points in each dimension
shape = (int(30e-3/spacing[0]),
         int(18e-3/spacing[1]))

# Define the origin coordinate, which is the top left corner of the grid
origin = (0., 0.)

# Compressional wave speed
vp_background = 1500  # Background speed in [m/s].
vp = np.full(shape, vp_background, dtype=np.float32)

# Density
rho_background = 1000  # Background density in [kg/m^3].
rho = np.full(shape, rho_background, dtype=np.float32)

- **Frequency-dependent Power Law Absorption**:
  - **$\alpha_0$**: 0.7 dB/(cm MHz^y)
  - **$y$**: 1.5

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define the physical fit function
def fit_function(w, eta_1, tau_1, eta_2, tau_2):
    term1 = (eta_1 / (2 * 1540 * tau_1)) * (w**2 * tau_1**2) / (1 + w**2 * tau_1**2)
    term2 = (eta_2 / (2 * 1540 * tau_2)) * (w**2 * tau_2**2) / (1 + w**2 * tau_2**2)
    return term1 + term2

# Generate w values from 0 to 2*pi*20 MHz
w = np.linspace(0, 2 * np.pi * 12e6, num=1000)

a0 = 0.7
y = 1.5
a_w = a0 * (w /(2*np.pi* 1e6))**y

# Refined initial guesses based on physical intuition
initial_guesses = [0.5, 1.e-8, 0.5, 1.0e-6]

optimal_parameters, covariance = curve_fit(
    fit_function, w, a_w, p0=initial_guesses, maxfev=100000
)

# Extract the optimal parameters
eta_1, tau_1, eta_2, tau_2 = optimal_parameters

# Print the results
print(f"Optimal eta_1: {eta_1}")
print(f"Optimal tau_1: {tau_1}")
print(f"Optimal eta_2: {eta_2}")
print(f"Optimal tau_2: {tau_2}")

# Compute the fitted a(w) using the optimal parameters
fitted_a_w = fit_function(w, eta_1, tau_1, eta_2, tau_2)

# Plot the original a(w) data and the fitted curve
plt.figure(figsize=(10, 6))
plt.plot(w/(2*np.pi*1e6), a_w, 'b-', label='Original $a(w)$')  # Original data
plt.plot(w/(2*np.pi*1e6), fitted_a_w, 'r--', label='Fitted $a(w)$')  # Fitted curve
plt.xlabel('Frequency (MHz)')
plt.ylabel('$a(w)$')
plt.title('Original vs Fitted $a(w)$')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
db2neper = 100 * (1.e-6 / (2*np.pi))**y / (20 * np.log10(np.exp(1)))

# Transfer from dB to Np
eta_1 = eta_1*db2neper*(2*np.pi* 1e6)**y
eta_2 = eta_2*db2neper*(2*np.pi* 1e6)**y

With the simulating grid and medium defined, we can create a 3D model with a time order of 2 and space order of 4. The size of boudary layers, also the perfectly matched layers, is 20 grid points.

In [ ]:
time_order = 2
space_order = 4
nbl = 20  # Number of boundary layers. Size of PML

# Define simulation time.
t0 = 0.0  # Start time of the simulation.
tn = 1.e-5  # End time of the simulation [s].
dt = 6e-10  # Time step [s].
time_range = TimeAxis(start=t0, stop=tn, step=dt)
nt = time_range.num

# Create a model
model = Model(vp=vp, rho=rho, origin=origin, shape=shape, spacing=spacing, space_order=space_order, dt=dt, nbl=nbl)


## Define the source

A line source transmits a 3-cycle tone burst plane wave at a central frequency of $2 \text{ MHz}$. The line source covers a range from $0$ to $30 \text{ mm}$ in x direction, with a fixed z-coordinate at $1 \text{ mm}$.

The source coordinates are defined out of grid and in the unit of meter. Here, for convenience, we make spacing between neighboring source points uniform and matching the grid spacing. The total number of source points is calculated by $(30 \text{ mm}/\text{spacing})$.


In [ ]:
# Define a line source.
f0 = 2e6 # Central frequency in [Hz]
src_npoints = int(30.e-3 / spacing[0])
src = ToneBurstSource(name='src', grid=model.grid, f0=f0, cycles=3, npoint=src_npoints, time_range=time_range) # source signal is a 3-cycle tone burst

# Define source coordinates
src.coordinates.data[:, 0] = np.arange(0., 30.e-3, spacing[0]) # x-direction
src.coordinates.data[:, 1] = 2.e-3 # z-direction

## Define the receiver
The definition of receiver is similar to source. We define four point receivers to receive the signals.

In [ ]:
# Define a point receiver
rec = Receiver(name='rec', grid=model.grid, npoint=4, time_range=time_range)
rec.coordinates.data[0, 0] = 15.e-3
rec.coordinates.data[0, 1] = 2.e-3
rec.coordinates.data[1, 0] = 15.e-3
rec.coordinates.data[1, 1] = 3.35e-3
rec.coordinates.data[2, 0] = 15.e-3
rec.coordinates.data[2, 1] = 8.75e-3
rec.coordinates.data[3, 0] = 15.e-3
rec.coordinates.data[3, 1] = 15.5e-3

In [ ]:
plot_velocity(model, source=src.coordinates.data, receiver=rec.coordinates.data)

## Operator
Operator acts as the computational engine of UltraWave. It takes the model, source, and receiver configurations as input and implement the wave equations.

In [ ]:
# Define the operator
op = Acoustic2DOperator(model, source=src, reciever=rec, eta_1=eta_1, eta_2=eta_2, tau_1=tau_1, tau_2=tau_2)

# Run the operator
op(time=time_range.num-1, dt=dt)

In [ ]:
RF0 = rec.data#[:, 1]
time = np.linspace(t0, tn, nt)
plt.figure(figsize=(12,8))
plt.plot(time, RF0, '-r')
#plt.plot(time, rec.data[:,2])
plt.xlabel('time (ms)')
plt.show()